<h1>Segmenting and Clustering Neighborhoods in Toronto<\h1>

All required libraries imported

In [85]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

Scraping data from wikipedia page into torr_canada_data and  extracting the table into My_table

In [86]:
torr_canada_data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(torr_canada_data,'lxml')
#print(soup.prettify())
My_table = soup.find('table',{'class':'wikitable sortable'})




Initialising the dataframe 

In [87]:
column_names=['PostalCode', 'Borough', 'Neighborhood']
neighborhoods = pd.DataFrame(columns=column_names)

Extracting the info from the table and storing in a dictionory to handle 'Not Assigned' and multiple neighborhood cases 
and writing the data into the data frame

In [88]:
neighbor_data=My_table.findAll('tr')
n_dict={}
i=0
for data in neighbor_data:
    if i==0:
        #to skip the headings
        i+=1
    else:
        postalcode=data.findAll('td')[0].text        
        borough=data.findAll('td')[1].text
        neighborhood_name=data.findAll('td')[2].text.split('\n')[0]
        
        if 'Not assigned' not in borough:
            if postalcode not in n_dict.keys():
                n_dict[postalcode]={}
                n_dict[postalcode]['borough']=borough
                if 'Not assigned' not in neighborhood_name:
                    n_dict[postalcode]['neighbor']=neighborhood_name
                else:
                    n_dict[postalcode]['neighbor']=borough
            else:
                n_dict[postalcode]['neighbor']+=', '+neighborhood_name
        
for p_code in n_dict:
    neighborhoods = neighborhoods.append({
                                          'PostalCode' : p_code,
                                          'Borough': n_dict[p_code]['borough'],
                                          'Neighborhood':n_dict[p_code]['neighbor'],
                                            }, ignore_index=True)
        
        
        

    
    







In [89]:
neighborhoods.shape

(103, 3)

In [90]:
neighborhoods

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Since I was not able to import geocoder I went with the csv file

In [91]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
temp_neighborhoods_df=neighborhoods.merge(coordinates,left_on='PostalCode',right_on='Postal Code')
final_neighborhoods_df=temp_neighborhoods_df.drop('Postal Code',axis=1)
final_neighborhoods_df



,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [92]:
final_neighborhoods_df.shape

(103, 5)

In [93]:
import folium
from geopy.geocoders import Nominatim
final_neighborhoods_df['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [94]:
for bor in final_neighborhoods_df['Borough']:
    if 'Toronto'.lower() in bor.lower():
        print(bor)

Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Downtown Toronto
West Toronto
Central Toronto
Downtown Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto


In [95]:
Central_Toronto= final_neighborhoods_df[final_neighborhoods_df['Borough'].str.contains('Toronto')].reset_index(drop=True)

Central_Toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


In [96]:
address = 'Central Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


In [97]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Central_Toronto['Latitude'], Central_Toronto['Longitude'], Central_Toronto['Borough'], Central_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto




In [98]:
CLIENT_ID = 'NJAVDFN0EVSI50EJLIW1EPT0WRUE1HUWQFWJAZOIZI2NG2AO' # your Foursquare ID
CLIENT_SECRET = 'YDZFP5IZ5XDDPN1UFNKXTZDBEJD23R25CUP1XPRBADVU0RA2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NJAVDFN0EVSI50EJLIW1EPT0WRUE1HUWQFWJAZOIZI2NG2AO
CLIENT_SECRET:YDZFP5IZ5XDDPN1UFNKXTZDBEJD23R25CUP1XPRBADVU0RA2


In [99]:
Central_Toronto.loc[0, 'Neighborhood']

'Harbourfront, Regent Park'

In [100]:
neighborhood_latitude =Central_Toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Central_Toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Central_Toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront, Regent Park are 43.6542599, -79.3606359.


In [101]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)


In [102]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [103]:
Central_Torronto_venues = getNearbyVenues(names=Central_Toronto['Neighborhood'],
                                   latitudes=Central_Toronto['Latitude'],
                                   longitudes=Central_Toronto['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [104]:
print(Central_Torronto_venues.shape)
Central_Torronto_venues.head()

(834, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [105]:
Central_Torronto_onehot = pd.get_dummies(Central_Torronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Central_Torronto_onehot['Neighborhood'] = Central_Torronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Central_Torronto_onehot.columns[-1]] + list(Central_Torronto_onehot.columns[:-1])
Central_Torronto_onehot = Central_Torronto_onehot[fixed_columns]

Central_Torronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
Central_Torronto_onehot.shape

(834, 187)

In [107]:
Ct_grouped = Central_Torronto_onehot.groupby('Neighborhood').mean().reset_index()
Ct_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.033333,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333


In [108]:
num_top_venues = 3

for hood in Ct_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Ct_grouped[Ct_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
        venue  freq
0  Steakhouse  0.10
1        Café  0.10
2       Hotel  0.07


----Berczy Park----
                venue  freq
0                Café  0.07
1        Cocktail Bar  0.07
2  Seafood Restaurant  0.07


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0     Coffee Shop   0.1
1            Café   0.1
2  Breakfast Spot   0.1


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.12
1  Gym / Fitness Center  0.06
2         Auto Workshop  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14


----Cabbagetown, St. James Town----
                venue  freq
0          Restaurant  0.10
1                Café  0.07
2  Italian Restaurant  0.07


----Central Bay Street----
             venue  fr

In [109]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [110]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Ct_grouped['Neighborhood']

for ind in np.arange(Ct_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Ct_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Steakhouse,Asian Restaurant,Hotel,Pizza Place,Noodle House,Concert Hall,Plaza,Monument / Landmark,Lounge
1,Berczy Park,Café,Seafood Restaurant,Cocktail Bar,Farmers Market,Bakery,Italian Restaurant,Restaurant,Bistro,Belgian Restaurant,Beer Bar
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym,Italian Restaurant,Climbing Gym,Convenience Store,Caribbean Restaurant,Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Spa,Skate Park,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop,Recording Studio
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boat or Ferry,Plane,Boutique


In [111]:

from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

Ct_grouped_clustering = Ct_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Ct_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [112]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Ct_merged = Central_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Ct_merged = Ct_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Ct_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Café,Pub,Mexican Restaurant,Breakfast Spot,Chocolate Shop,Restaurant,Dessert Shop
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Café,Clothing Store,Art Gallery,Sandwich Place,Shopping Mall,Beer Bar,Coffee Shop,Japanese Restaurant,Ramen Restaurant,Burger Joint
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Gastropub,Restaurant,Coffee Shop,Hotel,Italian Restaurant,Japanese Restaurant,BBQ Joint,Gym,Performing Arts Venue,Café
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Coffee Shop,Health Food Store,Pub,Cuban Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Café,Seafood Restaurant,Cocktail Bar,Farmers Market,Bakery,Italian Restaurant,Restaurant,Bistro,Belgian Restaurant,Beer Bar


In [113]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Ct_merged['Latitude'], Ct_merged['Longitude'], Ct_merged['Neighborhood'], Ct_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [114]:
Ct_merged.loc[Ct_merged['Cluster Labels'] == 0, Ct_merged.columns[[1] + list(range(5, Ct_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Café,Clothing Store,Art Gallery,Sandwich Place,Shopping Mall,Beer Bar,Coffee Shop,Japanese Restaurant,Ramen Restaurant,Burger Joint
2,Downtown Toronto,0,Gastropub,Restaurant,Coffee Shop,Hotel,Italian Restaurant,Japanese Restaurant,BBQ Joint,Gym,Performing Arts Venue,Café
4,Downtown Toronto,0,Café,Seafood Restaurant,Cocktail Bar,Farmers Market,Bakery,Italian Restaurant,Restaurant,Bistro,Belgian Restaurant,Beer Bar
6,Downtown Toronto,0,Grocery Store,Café,Park,Italian Restaurant,Convenience Store,Nightclub,Restaurant,Diner,Baby Store,Athletics & Sports
7,Downtown Toronto,0,Café,Steakhouse,Asian Restaurant,Hotel,Pizza Place,Noodle House,Concert Hall,Plaza,Monument / Landmark,Lounge
8,West Toronto,0,Pharmacy,Discount Store,Bakery,Supermarket,Music Venue,Brewery,Bar,Bank,Middle Eastern Restaurant,Liquor Store
9,Downtown Toronto,0,Park,Hotel,Café,Skating Rink,Salad Place,Sporting Goods Shop,Italian Restaurant,Basketball Stadium,Japanese Restaurant,Bistro
10,West Toronto,0,Bar,Asian Restaurant,Vietnamese Restaurant,Pizza Place,Yoga Studio,New American Restaurant,Boutique,Korean Restaurant,Record Shop,Bakery
11,East Toronto,0,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pizza Place,Cosmetics Shop,Pub,Restaurant,Dessert Shop,Diner
14,East Toronto,0,Sandwich Place,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Burrito Place,Burger Joint,Brewery,Liquor Store
